In [1]:
!pip install xgboost

In [2]:
#importing all necessary dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from urllib.parse import urlparse
from tld import get_tld
import re
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
import pickle
from flask import Flask, redirect, url_for, request, render_template

In [3]:
def firstDir(url):
    urlpath= urlparse(url).path
    try:
        return len(urlpath.split('/')[1])
    except:
        return 0
    
def tldf(tld):
    try:
        return len(tld)
    except:
        return -1  
    
def digitCount(url):
    digits = 0
    for i in url:
        if i.isnumeric():
            digits = digits + 1
    return digits

def letterCount(url):
    letters = 0
    for i in url:
        if i.isalpha():
            letters = letters + 1
    return letters

def numdir(url):
    urldir = urlparse(url).path
    return urldir.count('/')

In [4]:
def haveIP(url):
    match = re.search(
        '(([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.'
        '([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\/)|'  # IPv4
        '((0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\/)' # IPv4 in hexadecimal
        '(?:[a-fA-F0-9]{1,4}:){7}[a-fA-F0-9]{1,4}', url)  # Ipv6
    if match:
        # print match.group()
        return -1
    else:
        # print 'No matching pattern found'
        return 1
def shorteningurl(url):
    match = re.search('bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|'
                      'yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|'
                      'short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|'
                      'doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|'
                      'db\.tt|qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|'
                      'q\.gs|is\.gd|po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|'
                      'x\.co|prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|'
                      'tr\.im|link\.zip\.net',
                      url)
    if match:
        return -1
    else:
        return 1

In [ ]:
app = Flask(__name__, template_folder='./frontend', static_url_path='/frontend')

@app.route('/')
def hello_world():
    return render_template("index.html")


@app.route('/result', methods = ['GET', 'POST'])
def shortlist():
     if request.method == 'POST':
        url = request.form['websites']
        data = []
        data.append(len(url))        
        data.append(len(urlparse(url).netloc))
        data.append(len(urlparse(url).path))
        data.append(firstDir(url))
        try:
            data.append(len(get_tld(url,fail_silently=True)))
        except:
            data.append(0)
        data.append(tldf(get_tld(url,fail_silently=True)))
        data.append(url.count('-'))
        data.append(url.count('?'))
        data.append(url.count('%'))
        data.append(url.count('.'))
        data.append(url.count('='))
        data.append(url.count('http'))
        data.append(url.count('https'))
        data.append(url.count('www'))
        data.append(digitCount(url))
        data.append(letterCount(url))
        data.append(numdir(url))
        data.append(haveIP(url))
        data.append(shorteningurl(url))
        data = [0 if di is None else di for di in data]
        data=np.array(data).reshape(1, -1)
        with open("bestModel.pkl","rb") as f:
            model = pickle.load(f)
            f.close()
        prediction = np.int(model.predict(data))
        if prediction == 1:
            prediction = 'Real'
        else:
            prediction = 'Fake'
        
        binderList = prediction

        return render_template("shortlist.html", binderList=binderList, url = url)

    
if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [16/Aug/2023 14:48:22] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2023 14:48:22] "GET /favicon.ico HTTP/1.1" 404 -
/var/folders/8z/5jr2w07j661f129khmd4ym1m0000gn/T/ipykernel_22637/1604651000.py:40: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  prediction = np.int(model.predict(data))
/Users/muhammadmaaz/anaconda3/lib/python3.10/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warni

127.0.0.1 - - [16/Aug/2023 15:19:47] "GET / HTTP/1.1" 200 -
/var/folders/8z/5jr2w07j661f129khmd4ym1m0000gn/T/ipykernel_22637/1604651000.py:40: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  prediction = np.int(model.predict(data))
/Users/muhammadmaaz/anaconda3/lib/python3.10/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
127.0.0.1 - - [16/Aug/2023 15:19:54] "POST /result HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2023 15:20:08] "GET / HTTP/1.

In [ ]:
#edb5ade3